# Eval Vector Store Index 

---

---

# 1. Bedrock Client 생성

In [40]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/root/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


In [41]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Titan Embedding 모델 로딩

In [42]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

llm_emb = BedrockEmbeddings(client=boto3_bedrock)
llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f0a67c0c550>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

# 3. Index 이름 결정

In [43]:
index_name = "genai-poc-knox-cleaned-eval-sample-0.02-1024c-256o-v6"

chunk_size = 1024
chunk_overlap = 256


# 4. Load all Json files

In [6]:
from utils.proc_docs import get_load_json, show_doc_json

In [44]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/*.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

all json files:  1
all items:  1732


In [47]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.02, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  34


[Document(page_content='AR demo barcodes. Use the following barcodes to test the Knox Capture AR features. Barcode 1 Barcode 2 Barcode 3 Barcode 4 Barcode 5 Barcode 6', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'AR demo barcodes', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'}),
 Document(page_content="Videos. This section contains product and how-to videos related to Knox Capture. Getting started with Samsung Knox Capture In this video, we'll show you how to use Samsung Knox Capture to transform your mobile devices into powerful barcode scanners that can read, process, and output barcode data into other applications. Samsung Knox Capture: Enterprise-grade mobile scanning solution in Galaxy device This product intro video shows you how easy it is to transform rugged Samsung devices like the Galaxy XCover Pro into enterprise-grade barcode scanners.", metadata={'source': 'all_processe

# 5. Chunking JSON Doc 

In [48]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, SpacyTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size = chunk_size,
    chunk_overlap  = chunk_overlap,
    separators=["\n\n", "\n", ".", " ", ""],
    length_function = len,
)

chunk_docs = text_splitter.split_documents(eval_docs)
print(f"Number of chunk_docs after split and chunking= {len(chunk_docs)}")

Number of chunk_docs after split and chunking= 152


In [49]:
chunk_docs[40:50]

[Document(page_content='. Knox Capture pads the left side of the string with zeros until the desired length is reached. 1 Replace string &mdash; Replaces a specific string with a new, specified string. You must specify both strings. None Remove characters When sending data, removes a specific number of characters from the given positions. You must provide the start position and the number of characters to remove. Start position: 1 Number of characters: 1 Data sending Controls a selected amount of data to send to business apps. Send next characters &mdash; Sends a specific number of characters from the current cursor position. 1 Send remaining characters &mdash; Sends all data after the current cursor position. None Send up to substring &mdash; Sends all data up to a specific string. None Send pause &mdash; Pauses all actions for a specific number of milliseconds (max. 120000). 50ms Send string &mdash; Sends a specific string when the corresponding rule is triggered', metadata={'source'

# 4. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [50]:
from langchain.indexes import VectorstoreIndexCreator
# from langchain.vectorstores import FAISS

In [51]:
from utils.proc_docs import get_parameter

In [52]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [53]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [54]:
from utils.opensearch import opensearch_utils

In [55]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [56]:
from utils.opensearch import opensearch_utils

In [57]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )
else:
    print("Index does not exist")

index_name=genai-poc-knox-cleaned-eval-sample-0.01-1024c-256o-v6, exists=False
Index does not exist


In [58]:
from langchain.vectorstores import OpenSearchVectorSearch

In [59]:
%%time
# by default langchain would create a k-NN index and the embeddings would be ingested as a k-NN vector type
docsearch = OpenSearchVectorSearch.from_documents(
    index_name=index_name,
    documents=chunk_docs,
    embedding=llm_emb,
    opensearch_url=opensearch_domain_endpoint,
    http_auth=http_auth,
    bulk_size=100000,
    timeout=60
)

CPU times: user 443 ms, sys: 14.8 ms, total: 457 ms
Wall time: 14.8 s


## 인덱스 확인

In [60]:
index_info = os_client.indices.get(index=index_name)
pprint(index_info)

{'genai-poc-knox-cleaned-eval-sample-0.01-1024c-256o-v6': {'aliases': {},
                                                           'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                                   'project': {'fields': {'keyword': {'ignore_above': 256,
                                                                                                                                                      'type': 'keyword'}},
                                                                                                                               'type': 'text'},
                                                                                                                   'seq_num': {'type': 'long'},
                                                                                                                   'source': {'fields': {'keyword':

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [61]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2"
)
vector_db

In [62]:
# query = "knox"
query = "Configure keystroke output"
query = opensearch_utils.get_query(
    query=query
)

print("query: ", query)
response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

query:  {'query': {'bool': {'must': [{'match': {'text': {'query': 'Configure keystroke output', 'minimum_should_match': '0%', 'operator': 'or'}}}], 'filter': []}}}
# of searched docs:  10
# of display: 3
---------------------
_id in index:  5d7284bc-16fb-4fe0-b31e-34e7173821ba
7.4853153
Configure keystroke output and data formatting. Knox Capture sends scanned data to business apps as keystrokes, imitating the way a user manually enters data using a keyboard. If you activate a KPE (Knox Platform for Enterprise) Premium license with a Galaxy S20 device in WP-C mode, you must set `AdvancedRestrictionPolicy.allowRemoteControl()` to True in your EMM. Enabling this policy allows keystroke output to function as expected. Keystroke output Refer to the following settings to modify scanned data and format its output: Setting Description Options Default Send Enter character as Sets the output format for all existing and appended Enters Key event &mdash; Sends Enter as a key event. String &mdash;

# 5.오픈 서치에 "유사 서치" 검색
- query 를 제공해서 실제로 유사한 내용이 검색이 되는지를 확인 합니다.



In [63]:
from langchain.chains.question_answering import load_qa_chain
from utils.rag import get_semantic_similar_docs, get_lexical_similar_docs, get_ensemble_results

In [64]:
import copy
from langchain.schema import Document
from langchain import PromptTemplate
from operator import itemgetter

In [65]:
from utils.proc_docs import search_hybrid

In [66]:
%%time


filter01 = "[]"
filter02 = "[]"

query = "Configure keystroke output"

search_hybrid_result = search_hybrid(
    query=query,
    vector_db=vector_db,
    k=5,
    index_name= index_name,
    os_client=os_client,
    filter=[
        {"term": {"metadata.type": filter01}},
        {"term": {"metadata.source": filter02}},
    ],
    Semantic_Search = False,    
    Lexical_Search = False,    
    Hybrid_Search = True,     
    minimum_should_match = 75,   
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"]
    ensemble_weights=[.5, .5], # 시멘트 서치에 가중치 0.5 , 키워드 서치 가중치 0.5 부여.
    verbose=True
)



Query: 
 Configure keystroke output
##############################
similar_docs_semantic
##############################

Score: 1.0
['. Both String and Key event &mdash; Sends Enter as ASCII Line Feed 0x0A and key event. None &mdash; Sends no output for Enter. None Send Tab character as Sets the output format for all existing and appended Tabs Key event &mdash; Sends Tab as a key event. String &mdash; Sends Tab as ASCII Horizontal Tab 0x09. Both String and Key event &mdash; Sends Tab as ASCII Horizontal Tab 0x09 and key event. None &mdash; Sends no output for Tab. None Send Carriage Return character as Sets the output format for all Carriage Returns Key event &mdash; Sends Carriage Return as a key event. String &mdash; Sends Carriage Return as ASCII Carriage Return 0x0D. Both String and Key event &mdash; Sends Carriage Return as ASCII Carriage Return 0x0D and key event. None &mdash; Sends no output for Carriage Return. None Send ASCII Control characters as Sets the output format for AS